In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# DB 연결
import pymysql

# 실시간 주식가격 데이터
import FinanceDataReader as fdr

# 텍스트 분석
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from konlpy.tag import Okt
from gensim import models

from wordcloud import WordCloud

# 모델 학습 및 평가
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score

# 기타
import datetime
from collections import Counter
import sys
import warnings
warnings.filterwarnings('ignore')

In [2]:
df_all_lgc = pd.read_csv('df_youtube_삼프로_LG화학.csv')

In [3]:
df_all_lgc.head()

,st_n,st_cd,ch_nm,date,title,text,views,length,description,url
0,LG화학,51910,삼프로TV_경제의신과함께,2021090900,[증시 염탐정] 출발 분위기 안좋은 시장.. 삼성전자 흐름을 주목해보자 f.이베스트...,자 그러면 어저희와 함께 오늘 증시 개장 상황 이살펴 주실 염 블리 e 베스트투자증...,42597,1826,삼프로TV제작사및멤버사공개채용세부공고확인링크httpstlySR1n쿨썸머여름특가반값세...,https://www.youtube.com/watch?v=GT0dKiVKwGs
1,LG화학,51910,삼프로TV_경제의신과함께,2021090900,"[글로벌 이슈체크] 수요 꺾이지 않은 글로벌 배터리, 비중 확대해도 괜찮을까? f...",글로벌 iv 부 시작하도록 하겠습니다아니 오늘 및 미래 3 측에서새식구를 보내고 나...,17745,1556,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=3XFFyz8z_hE
2,LG화학,51910,삼프로TV_경제의신과함께,2021090900,"투자 전략 전문가가 바라보는 국내 증시 방향성은?_박병창, 한지영, 염승환",[음악]으no[음악]cool합니다[음악]아 으[음악]아으[박수][음악]기피하는 경제...,309214,7095,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=F1bb9QiGzKY
3,LG화학,51910,삼프로TV_경제의신과함께,2021090800,"[뉴스3] ""美증시 비중축소""…잇단 비관론에 다우 270p↓",자 오늘 9월 8일 수요일 순서와먼저 뉴스 3부터 내 무게 좀 정리를해 드릴 텐데 ...,15483,1348,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=4HZCWvZpp6U
4,LG화학,51910,삼프로TV_경제의신과함께,2021090900,[심층 인터뷰] 추석과 FOMC.. 포트폴리오의 조정이 필요하다? f.키움증권 한지...,자 그러면 오늘 아침 아 김 증권의한지영 아예 무영과 함께 우리전략을 한번 노래하고...,14930,1570,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=zb2heNm_sjM


In [65]:
df_all_lgc[df_all_lgc['date'] == 2021090900]

,st_n,st_cd,ch_nm,date,title,text,views,length,description,url
0,LG화학,51910,삼프로TV_경제의신과함께,2021090900,[증시 염탐정] 출발 분위기 안좋은 시장.. 삼성전자 흐름을 주목해보자 f.이베스트...,자 그러면 어저희와 함께 오늘 증시 개장 상황 이살펴 주실 염 블리 e 베스트투자증...,42597,1826,삼프로TV제작사및멤버사공개채용세부공고확인링크httpstlySR1n쿨썸머여름특가반값세...,https://www.youtube.com/watch?v=GT0dKiVKwGs
1,LG화학,51910,삼프로TV_경제의신과함께,2021090900,"[글로벌 이슈체크] 수요 꺾이지 않은 글로벌 배터리, 비중 확대해도 괜찮을까? f...",글로벌 iv 부 시작하도록 하겠습니다아니 오늘 및 미래 3 측에서새식구를 보내고 나...,17745,1556,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=3XFFyz8z_hE
2,LG화학,51910,삼프로TV_경제의신과함께,2021090900,"투자 전략 전문가가 바라보는 국내 증시 방향성은?_박병창, 한지영, 염승환",[음악]으no[음악]cool합니다[음악]아 으[음악]아으[박수][음악]기피하는 경제...,309214,7095,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=F1bb9QiGzKY
4,LG화학,51910,삼프로TV_경제의신과함께,2021090900,[심층 인터뷰] 추석과 FOMC.. 포트폴리오의 조정이 필요하다? f.키움증권 한지...,자 그러면 오늘 아침 아 김 증권의한지영 아예 무영과 함께 우리전략을 한번 노래하고...,14930,1570,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=zb2heNm_sjM
5,LG화학,51910,삼프로TV_경제의신과함께,2021090900,[제길선생의 미국 돋보기] 3대 지수 약보합 마감.. 종목 장세 유지 중? f.교보...,jr 6시 55분에 저희 스튜디오를찾아 주신 우리 교보 지구의 박병철부장님 모두 정...,19036,1498,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=mijZ0CHbWVk
7,LG화학,51910,삼프로TV_경제의신과함께,2021090900,"[글로벌마켓 브리핑] 미국 경제지표 부진, 9월 글로벌 시장 전망은? f. 미래에...",일부 시작하도록 하겠습니다 예 jy님 언제든지 의도로 놀라워서 저 타워어 파크 원 ...,11165,1882,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=7lVJk-_URaI
11,LG화학,51910,삼프로TV_경제의신과함께,2021090900,배터리 산업의 주가를 결정하는 중요한 요소는? _글로벌 라이브 _글로벌 라이브_강구...,[음악][음악]으으 으으 으[음악]글로벌 라이브 에 오신걸 환영합니다안녕하십니까 박...,104076,6155,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=ZBM7yFbX93o
12,LG화학,51910,삼프로TV_경제의신과함께,2021090900,[증시 염탐정] 출발 분위기 안좋은 시장.. 삼성전자 흐름을 주목해보자 f.이베스트...,자 그러면 어저희와 함께 오늘 증시 개장 상황 이살펴 주실 염 블리 e 베스트투자증...,42597,1826,삼프로TV제작사및멤버사공개채용세부공고확인링크httpstlySR1n쿨썸머여름특가반값세...,https://www.youtube.com/watch?v=GT0dKiVKwGs
13,LG화학,51910,삼프로TV_경제의신과함께,2021090900,"[글로벌 이슈체크] 수요 꺾이지 않은 글로벌 배터리, 비중 확대해도 괜찮을까? f...",글로벌 iv 부 시작하도록 하겠습니다아니 오늘 및 미래 3 측에서새식구를 보내고 나...,17745,1556,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=3XFFyz8z_hE
14,LG화학,51910,삼프로TV_경제의신과함께,2021090900,"투자 전략 전문가가 바라보는 국내 증시 방향성은?_박병창, 한지영, 염승환",[음악]으no[음악]cool합니다[음악]아 으[음악]아으[박수][음악]기피하는 경제...,309214,7095,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=F1bb9QiGzKY


In [66]:
df_all_lgc[df_all_lgc['date'] == 2021090900].shape

(14, 10)

In [67]:
df_all_lgc[df_all_lgc['date'] == 2021090900].shape

(14, 10)

In [68]:
df_all_lgc[df_all_lgc['date'] == 2021090700].shape

(0, 10)

In [69]:
df_all_lgc[df_all_lgc['date'] == 2021090600].shape

(0, 10)

In [71]:
df_all_lgc[df_all_lgc['date'] == 2021090500].shape

(0, 10)

In [72]:
df_all_lgc[df_all_lgc['date'] == 2021090400].shape

(0, 10)

In [73]:
df_all_lgc[df_all_lgc['date'] == 2021090300].shape

(0, 10)

In [4]:
# description, url 제외
df_lgc = df_all_lgc.iloc[:, :8]

In [5]:
df_lgc.head()

,st_n,st_cd,ch_nm,date,title,text,views,length
0,LG화학,51910,삼프로TV_경제의신과함께,2021090900,[증시 염탐정] 출발 분위기 안좋은 시장.. 삼성전자 흐름을 주목해보자 f.이베스트...,자 그러면 어저희와 함께 오늘 증시 개장 상황 이살펴 주실 염 블리 e 베스트투자증...,42597,1826
1,LG화학,51910,삼프로TV_경제의신과함께,2021090900,"[글로벌 이슈체크] 수요 꺾이지 않은 글로벌 배터리, 비중 확대해도 괜찮을까? f...",글로벌 iv 부 시작하도록 하겠습니다아니 오늘 및 미래 3 측에서새식구를 보내고 나...,17745,1556
2,LG화학,51910,삼프로TV_경제의신과함께,2021090900,"투자 전략 전문가가 바라보는 국내 증시 방향성은?_박병창, 한지영, 염승환",[음악]으no[음악]cool합니다[음악]아 으[음악]아으[박수][음악]기피하는 경제...,309214,7095
3,LG화학,51910,삼프로TV_경제의신과함께,2021090800,"[뉴스3] ""美증시 비중축소""…잇단 비관론에 다우 270p↓",자 오늘 9월 8일 수요일 순서와먼저 뉴스 3부터 내 무게 좀 정리를해 드릴 텐데 ...,15483,1348
4,LG화학,51910,삼프로TV_경제의신과함께,2021090900,[심층 인터뷰] 추석과 FOMC.. 포트폴리오의 조정이 필요하다? f.키움증권 한지...,자 그러면 오늘 아침 아 김 증권의한지영 아예 무영과 함께 우리전략을 한번 노래하고...,14930,1570


In [8]:
# 출처 '자체제작 감성어휘사전 모듈_ver2.ipynb'

def set_data(df,com_num = 0):  # panda, numpy, datetime, FinanceDataReader, konlpy, Counter
    # 일, 시간 나누기
    df['date'] = df['date'].astype(str)
    df['date_d'] = df['date'].str[:-2]
    df['date_h'] = df['date'].str[-2:]
    # 타입을 데이트 타입으로 만듬
    df['date_d'] = pd.to_datetime(df['date_d'])
    
    df = df.sort_values(by='date_d') # 일기준으로 오름차순 정렬
    
    if com_num != 0:
    #     통합 데이터 활용시 업종코드 지정
        df = df[df['st_cd'] == com_num] # 해당 회사만 추출
        num = str(com_num).zfill(6) # 종목코드를 6자리로 만들어줌
        
    else:
        # 특정 업종만 할 때
        num = str(df['st_cd'].iloc[0]).zfill(6)
    
    # 널값 제거
    df = df.dropna()
    
    # ============== 날짜 조정
    ## 전일 15시 ~ 금일 15시로 날짜 조정
    after_market = ['15', '16', '17', '18', '19', '20', '21', '22', '23']
    
    for i in range(len(df)):
        if df['date_h'].iloc[i] in after_market:
            df['date_d'].iloc[i] += datetime.timedelta(1)
    
    # 주말 및 공휴일 데이터
    ### Holidays
    try:
        # 서버가 열려있을 때
        db = pymysql.connect(user='root',
                             passwd='1234',
                             host='3.35.70.166',
                             db='proj',
                             charset='utf8')

        cursor = db.cursor(pymysql.cursors.DictCursor)

        sql = "select * from holidays"
        cursor.execute(sql)
        result = cursor.fetchall()
    
        # DataFrame으로 변경
        holi = pd.DataFrame(result)
        # db 닫기 --> 안하면 메모리 잡아먹음
        db.close()
    except:
        # 서버 없을 때 깃허브에서 바로 가져옴
        db_holi = 'https://raw.githubusercontent.com/chaerui7967/stock_predict_news_and_youtube/master/%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D/%EC%97%B0%EC%8A%B5%20%EB%8D%B0%EC%9D%B4%ED%84%B0/holidays.csv'
        holi = pd.read_csv(db_holi)
    
    # date 컬럼을 날짜 형식으로 변경
    holi['date'] = pd.to_datetime(holi['date'])
    
    
    ### ===================주말 및 공휴일 제외
    
    ## 주말 및 공휴일만 추출
    market_closed = holi[holi['holiday']=="O"].reset_index(drop=True)
    
    ## 휴장일 List 생성
    market_closed_list = list(market_closed['date'])
    
    # 주말 및 공휴일 제외
    while len(df[df['date_d'].isin(market_closed_list)]['date_d']) !=0:
        for i in df[df['date_d'].isin(market_closed_list)]['date_d'].index:
            df['date_d'][i] += datetime.timedelta(1)
     
    
    
    # 주식 데이터 가져오기
    start_date = str(df['date_d'].iloc[0])[:10].replace('-', '')
    end_date = str(df['date_d'].iloc[-1])[:10].replace('-', '')  # 뉴스 데이터 기준 최근 값까지의 주식 데이터
    
    stock = fdr.DataReader(num, start = start_date, end = end_date)
    
    # change 컬럼 판단 기준 close에서 수수료 보다 많은 비율이 올랐으면 1, 떨어졌으면 -1, 그 사이 0
    point = stock['Close'] * stock['Change']
    spli = stock['Close'] * 0.001  # 수수료가 0.1 % 라고 할 때
    
    stock['ud'] = np.where( point > spli, 1,
                          np.where(point < spli, -1, 0))
    
    stock = stock.reset_index() # 날짜 컬럼 가져오기
    
    s_1 = stock[['Date','ud']] # 필요한 컬럼만 가져오기
    
    s_1.columns = ['date_d','ud'] # 컬럼 명 초기화
    
    # 뉴스데이터와 주식데이터 merge
    df_1 = pd.merge(df,s_1,on='date_d')  # 영향받는 컬럼 기준으로 inner 조인 
    
    return df_1

In [20]:
dataSet = set_data(df_lgc)
dataSet.head()

,st_n,st_cd,ch_nm,date,title,text,views,length,date_d,date_h,ud
0,LG화학,51910,삼프로TV_경제의신과함께,2021090800,[증시 염탐정] 약보합으로 출발한 시장.. 대형주의 방향은? f.이베스트투자증권 염...,자 그러면 오늘 또 증시 개장 상 1정리 해 주시아 염수 아니라 니모 지겠습니다 어...,100475,1813,2021-09-08,00,-1
1,LG화학,51910,삼프로TV_경제의신과함께,2021090800,"[뉴스3] ""美증시 비중축소""…잇단 비관론에 다우 270p↓",자 오늘 9월 8일 수요일 순서와먼저 뉴스 3부터 내 무게 좀 정리를해 드릴 텐데 ...,15483,1348,2021-09-08,00,-1
2,LG화학,51910,삼프로TV_경제의신과함께,2021090800,"[글로벌마켓 브리핑] 스가 총리 퇴진 발표 이후, 일본 증시 상승 랠리! f. 미래...",여러분 안녕하세요 광고 보고 왔습니다오늘 힘차게 한번 시작해 보도록하겠습니다 언제나...,7231,1711,2021-09-08,00,-1
3,LG화학,51910,삼프로TV_경제의신과함께,2021090800,[증시 셔터맨] 외인과 기관은 왜 삼성전자와 카카오뱅크를 매도할까? f. 한국투자증...,4 아 오늘한국투자증권 에서 오셨죠 우리박지영 차장을 모시겠습니다 어서오십시오 네 ...,21472,1511,2021-09-08,00,-1
4,LG화학,51910,삼프로TV_경제의신과함께,2021090800,[증시 셔터맨] 외인과 기관은 왜 삼성전자와 카카오뱅크를 매도할까? f. 한국투자증...,4 아 오늘한국투자증권 에서 오셨죠 우리박지영 차장을 모시겠습니다 어서오십시오 네 ...,21472,1511,2021-09-08,00,-1


In [21]:
dataSet.title[0]

'[증시 염탐정] 약보합으로 출발한 시장.. 대형주의 방향은? f.이베스트투자증권 염승환 이사'

In [22]:
len(dataSet.text[0])

12909

In [23]:
dataSet.text[0]

'자 그러면 오늘 또 증시 개장 상 1정리 해 주시아 염수 아니라 니모 지겠습니다 어서오십시오 네마 같습니까자 우리 여기서 님 저희가 한번 무슨얘기를 하든 걔도 모습은 앞에 이런모습 그 일로 모습 모집하고 있구요밑에도 노래 아이오와 따가 못 해마다픽으로 달라는 겁니다 우리도 그위험부담은 못하니까 l 합니다 이렇게때 못하며 예맥 에 강한 분들도 좀 요청해 주시기바라겠습니다 아김택진 진영이와 comte 머신 너무짜다예매트 뭐 저희가 참고하겠습니다 예 자우리 어머님과 함께 오늘도 증시 개장상황을 살펴 볼 텐데 오늘 한 10분정도 남았구요 먼저 그러면 어 리프트들 좀 보고 넘어갈까요 4으첫번째로는 요새 문은 행주 들도 좀많이 관심 들은 없긴한데 크긴 크지만좋고 일단 하나금융의 재정 옥애널리스트 그 은행 업종을 좀 되게간격으로 보고 있는 것 같아요 내간단히 말씀드리면 지금 6월 이후로그 미국의 10년 금리가 좀 빠지면서은행 제도 정확하게 동행하면서좀 주가가 부진하다 이렇게 좀 그최근 흐름을 좀 적어 줬고요 담아이제 긍정적으로 볼 부분들이 좀높아졌다 일단 크로나 금융지원 추가연장 일단 이렇게 졌기 때문에 추가충당금 적립 가능성 일단 조금낮아졌다고 합니다 그리고 3개월 동안외국인 순매도 가 계속 좀 이어졌는데일단 이게 멈췄고한 행 이제 코스피 200 때문에은행들 좀 수급 무료가 좀 있었는데이것도 이제 이번주 목요일 지만 사실끝나고 요그 다음에 여기에 8월 기준금리 인상연내 추가 인상 가능성도 높아진상황이니까은행주 실적 컨센서스 는 계속 상황이될 수 있다 요런 좀 내용이고 또테이퍼링 시계도 좀 논란은 있지만연주 4 긴축 속도가 좀 만 하다면경기 확장 북면 은 더 연장될가능성이 있다 그래서 다시한번 좀경기와 금리의 베팅할 시간이 좀돌아왔다요즘 내용이고 그래서 실적 상황에따라서주 당 대 당 금도 올라갈 가능성이높아졌기 때문에 4장 투자 수요도증가할 가능성이 있다고 합니다 그래서일단 변경된 은행 그 전체순위 이게일단 18.5 좋은 정도로 작년 대비해서 5% 가 내년도 순위 이끌 

In [25]:
dataSet.shape

(27, 11)

In [27]:
sample = dataSet.head()
sample

,st_n,st_cd,ch_nm,date,title,text,views,length,date_d,date_h,ud
0,LG화학,51910,삼프로TV_경제의신과함께,2021090800,[증시 염탐정] 약보합으로 출발한 시장.. 대형주의 방향은? f.이베스트투자증권 염...,자 그러면 오늘 또 증시 개장 상 1정리 해 주시아 염수 아니라 니모 지겠습니다 어...,100475,1813,2021-09-08,00,-1
1,LG화학,51910,삼프로TV_경제의신과함께,2021090800,"[뉴스3] ""美증시 비중축소""…잇단 비관론에 다우 270p↓",자 오늘 9월 8일 수요일 순서와먼저 뉴스 3부터 내 무게 좀 정리를해 드릴 텐데 ...,15483,1348,2021-09-08,00,-1
2,LG화학,51910,삼프로TV_경제의신과함께,2021090800,"[글로벌마켓 브리핑] 스가 총리 퇴진 발표 이후, 일본 증시 상승 랠리! f. 미래...",여러분 안녕하세요 광고 보고 왔습니다오늘 힘차게 한번 시작해 보도록하겠습니다 언제나...,7231,1711,2021-09-08,00,-1
3,LG화학,51910,삼프로TV_경제의신과함께,2021090800,[증시 셔터맨] 외인과 기관은 왜 삼성전자와 카카오뱅크를 매도할까? f. 한국투자증...,4 아 오늘한국투자증권 에서 오셨죠 우리박지영 차장을 모시겠습니다 어서오십시오 네 ...,21472,1511,2021-09-08,00,-1
4,LG화학,51910,삼프로TV_경제의신과함께,2021090800,[증시 셔터맨] 외인과 기관은 왜 삼성전자와 카카오뱅크를 매도할까? f. 한국투자증...,4 아 오늘한국투자증권 에서 오셨죠 우리박지영 차장을 모시겠습니다 어서오십시오 네 ...,21472,1511,2021-09-08,00,-1


## Stop Words List

In [87]:
# Update 계속 하는 중!
media = ['오늘', '년도', '이제', '요즘', '지금', '우리나라', '한번', '사실',  \
         '증시', '개장', '얘기', '이렇다', '참고', '없다', '때문', \
         '아니다', '어서', '오십', '시오', '모습', '내용', '정도', '그렇다', '어쨌든', '어떻다', \
         '박지영', '미래에셋', '어머님', '음악']

In [142]:
# 출처 '자체제작 감성어휘사전 모듈_ver2.ipynb' > def giro_dic
##표시 수정 부분

def youtube_pre(df,num = 0):
    
## media 전역 변수 선언
    global media
    
    df = set_data(df, num)
    print('데이터 셋 구성 완료')
    
    okt = Okt()
    print('okt 모듈 활성')
    
    
    # 데이터 전처리
    # =================== nlp
    # 1. 제목, 텍스트 --> 명사, 형용사 추출
    # 태그 삭제        
    # \n \t 삭제
    df['text'] = df['text'].str.replace('[\n|\t|\r]','')
    df['text'] = df['text'].str.replace('\s','')
    df['text'] = df['text'].str.replace('[^a-zA-Z0-9가-힣]','')
    
    # 토큰화 및 품사 태깅
    df['Contents_input'] = df['text'].apply(lambda x: okt.pos(x, norm=True, stem=True))
    print('태그 삭제 및 품사 태깅 완료')
    
    # 명사, 형용사 추출
    for i in range(len(df['Contents_input'])):
        wordList = []
        for j in range(len(df['Contents_input'][i])):
            if df['Contents_input'][i][j][1] == 'Noun' or df['Contents_input'][i][j][1] == 'Adjective':
                wordList.append(df['Contents_input'][i][j][0])         
        df['Contents_input'][i] = wordList
    print('명사, 형용사 추출')
    
    # 2. 불용어 처리
    # 불용어 사전 로딩
##     url = 'https://raw.githubusercontent.com/chaerui7967/stock_predict_news_and_youtube/master/%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D/data/stopwords_ver1.txt'
    url = 'https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt'
    stopwords = list(pd.read_csv(url, header=None)[0])
##     media = ['매일경제','mkcokr','무단전재및재배포금지','무단전재','재배포금지']
    media = ['오늘', '년도', '이제', '요즘', '지금', '우리나라', '한번', '사실', '처럼', \
             '증시', '개장', '얘기', '이렇다', '참고', '없다', '때문', '아니다', \
             '안녕하십니까', '안녕하세요우', '안녕하다', '어서', '오십', '시오', '모습', '내용', '정도', \
             '그렇다', '어쨌든', '어떻다', '박지영', '미래에셋', '어머님', '음악']
    print('불용어 사전 로딩..')
    
    
    for i in range(len(df['Contents_input'])):
        for word in df['Contents_input'][i]:
            if word in stopwords:
                df['Contents_input'][i].remove(word)
            if word in media:
                df['Contents_input'][i].remove(word) 
    print('불용어 처리 완료')
    
    # 단어길이가 1이하인 단어 제거
    for index, wordList in enumerate(df['Contents_input']):
        if len(df['Contents_input'][index]) == 0:
            continue
        for word in wordList:
            if len(word) <= 1:
                df['Contents_input'][index].remove(word)
    print('단어 길이 1이하인 단어 제거')

    return df

In [44]:
'''
# 단어 길이 1 이하 제거 안 되는 경우가 있음

for index, wordList in enumerate(sample_clean['Contents_input']):
    print(index, wordList)
    if len(sample_clean['Contents_input'][index]) == 0:
        continue
    for word in wordList:
        if len(word) <= 1:
            sample_clean['Contents_input'][index].remove(word)

sample_clean['Contents_input'][1] # 1 index '말'
'''

## Text-Preprocessing

In [56]:
# df_clean = youtube_pre(df)
# df_clean.head()

### Sample

In [55]:
%%time 

# 5개 샘플 데이터
sample_clean = youtube_pre(sample)

데이터 셋 구성 완료
okt 모듈 활성
태그 삭제 및 품사 태깅 완료
명사, 형용사 추출
불용어 사전 로딩..
불용어 처리 완료
단어 길이 1이하인 단어 제거
Wall time: 1min 16s


,st_n,st_cd,ch_nm,date,title,text,views,length,date_d,date_h,ud_x,ud_y,Contents_input
0,LG화학,51910,삼프로TV_경제의신과함께,2021090800,[증시 염탐정] 약보합으로 출발한 시장.. 대형주의 방향은? f.이베스트투자증권 염...,자그러면오늘또증시개장상1정리해주시아염수아니라니모지겠습니다어서오십시오네마같습니까자우리...,100475,1813,2021-09-08,00,-1,-1,"[정리, 염수, 니모, 오십, 네마, 한번, 하든, 일로, 모습, 모집, 노래, 아..."
1,LG화학,51910,삼프로TV_경제의신과함께,2021090800,"[뉴스3] ""美증시 비중축소""…잇단 비관론에 다우 270p↓",자오늘9월8일수요일순서와먼저뉴스3부터내무게좀정리를해드릴텐데예아무래도미국소식도좀갤주너...,15483,1348,2021-09-08,00,-1,-1,"[무게, 정리, 미국, 소식, 주너, 미국, 시장, 음소, 간밤, 진행, 좋다, 아..."
2,LG화학,51910,삼프로TV_경제의신과함께,2021090800,"[글로벌마켓 브리핑] 스가 총리 퇴진 발표 이후, 일본 증시 상승 랠리! f. 미래...",여러분안녕하세요광고보고왔습니다오늘힘차게한번시작해보도록하겠습니다언제나처럼미래에셋증권의...,7231,1711,2021-09-08,00,-1,-1,"[안녕하다, 광고, 힘차다, 한번, 시작, 처럼, 증권, 의장, 선임, 전이, 매년..."
3,LG화학,51910,삼프로TV_경제의신과함께,2021090800,[증시 셔터맨] 외인과 기관은 왜 삼성전자와 카카오뱅크를 매도할까? f. 한국투자증...,4아오늘한국투자증권에서오셨죠우리박지영차장을모시겠습니다어서오십시오네안녕하십니까4한국투...,21472,1511,2021-09-08,00,-1,-1,"[한국, 투자, 증권, 박지영, 차장, 시오, 안녕하십니까, 한국, 투자, 증권, ..."
4,LG화학,51910,삼프로TV_경제의신과함께,2021090800,[증시 셔터맨] 외인과 기관은 왜 삼성전자와 카카오뱅크를 매도할까? f. 한국투자증...,4아오늘한국투자증권에서오셨죠우리박지영차장을모시겠습니다어서오십시오네안녕하십니까4한국투...,21472,1511,2021-09-08,00,-1,-1,"[한국, 투자, 증권, 박지영, 차장, 시오, 안녕하십니까, 한국, 투자, 증권, ..."


In [ ]:
sample_clean

In [38]:
sample_clean.Contents_input[0]

['정리',
 '염수',
 '니모',
 '오십',
 '네마',
 '한번',
 '하든',
 '모습',
 '모습',
 '일로',
 '모습',
 '모집',
 '노래',
 '아이오와',
 '따갑다',
 '달라',
 '위험부담',
 '예맥',
 '강하다',
 '요청',
 '김택진',
 '진영',
 '머신',
 '매트',
 '어머님',
 '상황',
 '은행',
 '관심',
 '좋다',
 '금융',
 '재정',
 '옥애널',
 '리스트',
 '은행',
 '업종',
 '간격',
 '보고',
 '간단',
 '지금',
 '월동',
 '외국인',
 '매도',
 '계속',
 '코스피',
 '은행',
 '수급',
 '무료',
 '좋다',
 '목요일',
 '사실',
 '그다음',
 '긴축',
 '속도',
 '경기',
 '확장',
 '북면',
 '연장',
 '가능성',
 '다시',
 '한번',
 '경기',
 '금리',
 '베팅',
 '내용',
 '실적',
 '상황',
 '서주',
 '당대',
 '금도',
 '가능성',
 '투자',
 '수요',
 '증가',
 '가능성',
 '고합',
 '다그',
 '래서',
 '변경',
 '은행',
 '전체',
 '순위',
 '좋다',
 '정도',
 '작년',
 '대비',
 '가내',
 '순위',
 '정도',
 '증가',
 '예상',
 '그다음',
 '올해',
 '예상',
 '배당',
 '수익률',
 '지금',
 '정도',
 '지금',
 '투자',
 '행주',
 '연말',
 '은행',
 '대해',
 '관심',
 '사실',
 '지금',
 '좀뭐',
 '리스크',
 '만화',
 '한번',
 '접근',
 '그렇다',
 '요좀',
 '해주시',
 '좋다',
 '그다음',
 '증권',
 '재운',
 '이분',
 '사실',
 '우리나라',
 '굉장하다',
 '피트',
 '코인',
 '보고서',
 '비트코인',
 '가기',
 '급락',
 '사실',
 '최근',
 '설번',
 '거기',
 '법정',
 '놈',
 '개널',
 '비트코인',
 '법정',
 '통화',
 '채

In [39]:
sample_clean.Contents_input[1] # 한 글자 안 지워짐

['무게',
 '정리',
 '미국',
 '소식',
 '주너',
 '미국',
 '시장',
 '음소',
 '간밤',
 '진행',
 '좋다',
 '아대',
 '의심',
 '지수',
 '전일',
 '제전',
 '게제',
 '주말',
 '매년',
 '노동',
 '추구',
 '휴가',
 '자궁',
 '퍼센트',
 '가내',
 '마감',
 '다우지수',
 '좀더',
 '가내',
 '마감',
 '고기술',
 '심의',
 '스타',
 '터그',
 '포인트',
 '빛바랜',
 '사상',
 '철학',
 '어쨌든',
 '상체',
 '상처',
 '에덴',
 '맏형',
 '형파',
 '로우',
 '감안',
 '좀빛',
 '이바',
 '어제',
 '특별하다',
 '국채',
 '금리',
 '크게',
 '치질',
 '안제',
 '계속',
 '정도',
 '출판',
 '최근',
 '최근',
 '주간',
 '가장',
 '상승',
 '국회',
 '금리',
 '대체로',
 '증권사',
 '미국',
 '시장',
 '전망',
 '조그마하다',
 '캐드',
 '지난주',
 '싹수',
 '안되다',
 '의원',
 '바이러스',
 '확산',
 '기부',
 '체약',
 '유로',
 '전망',
 '하향',
 '조정',
 '올해',
 '성장',
 '전망',
 '포인트',
 '해연',
 '경제성장률',
 '도시',
 '컨센서스',
 '시장',
 '컨센서스',
 '통한',
 '다르다',
 '연간',
 '어떻다',
 '비교',
 '에이',
 '경기',
 '대한',
 '하향',
 '조정',
 '일수',
 '논평',
 '향후',
 '강력하다',
 '소비',
 '성장',
 '대한',
 '장애물',
 '높다',
 '이유',
 '델타',
 '초읽기',
 '인프라',
 '모에',
 '수시',
 '그렇다',
 '서예',
 '진짜',
 '역대',
 '데이',
 '사실',
 '털실',
 '국회',
 '계속',
 '민주당',
 '내부',
 '사실',
 '위원',
 '돌출',
 '그렇다',
 '상황',
 '네네',
 '래서',
 '정상'

In [57]:
count = Counter(sample_clean.Contents_input[0])
words = dict(count.most_common())
words

{'좋다': 28,
 '이제': 16,
 '중국': 14,
 '한번': 11,
 '사실': 11,
 '거기': 11,
 '기업': 11,
 '오늘': 11,
 '비트코인': 10,
 '하락': 10,
 '한국': 9,
 '일단': 9,
 '현대차': 9,
 '굉장하다': 8,
 '최근': 8,
 '네이버': 8,
 '어제': 8,
 '신흥': 8,
 '정도': 8,
 '은행': 7,
 '보고서': 7,
 '부분': 7,
 '반등': 7,
 '카카오': 7,
 '가능성': 6,
 '수수료': 6,
 '메모리': 6,
 '펀드': 6,
 '수소': 6,
 '그다음': 5,
 '투자': 5,
 '해주시': 5,
 '증권': 5,
 '우리나라': 5,
 '회사': 5,
 '높다': 5,
 '생각': 5,
 '우려': 5,
 '지수': 5,
 '관련': 5,
 '스트림': 5,
 '같다': 5,
 '음악': 5,
 '우리': 5,
 '강하다': 4,
 '리스트': 4,
 '업종': 4,
 '보고': 4,
 '계속': 4,
 '코스피': 4,
 '경기': 4,
 '대해': 4,
 '제품': 4,
 '거래': 4,
 '확산': 4,
 '미국': 4,
 '많다': 4,
 '긍정': 4,
 '나쁘다': 4,
 '주가': 4,
 '보시': 4,
 '말씀': 4,
 '뉴스': 4,
 '그게': 4,
 '여기': 4,
 '뭐': 4,
 '모습': 3,
 '상황': 3,
 '수급': 3,
 '속도': 3,
 '확장': 3,
 '증가': 3,
 '고합': 3,
 '다그': 3,
 '급락': 3,
 '법정': 3,
 '채택': 3,
 '시작': 3,
 '국민': 3,
 '매물': 3,
 '도로': 3,
 '이제우': 3,
 '가지': 3,
 '부담': 3,
 '자체': 3,
 '수로': 3,
 '네트워크': 3,
 '경우': 3,
 '솔루션': 3,
 '라이트닝': 3,
 '참고': 3,
 '공개': 3,
 '삼성': 3,
 '단기': 3,
 '가격': 3,
 '당연하다': 3,
 '영

In [62]:
words_df = pd.DataFrame.from_dict([words]).T.reset_index()
words_df.columns = ['words', 'freq']

In [63]:
words_df

,words,freq
0,좋다,28
1,이제,16
2,중국,14
3,한번,11
4,사실,11
...,...,...
904,강의,1
905,할인,1
906,수강,1
907,페이지,1


In [64]:
words_df.to_csv('sample_wc.csv', header=True, index=False, encoding = 'utf-8-sig')

### 날짜별 버전

In [89]:
# lg chemical [날짜] before

# lgc_210908_bf = dataSet[dataSet['date_d'] == '2021-09-08']
# lgc_210908_bf

,st_n,st_cd,ch_nm,date,title,text,views,length,date_d,date_h,ud
0,LG화학,51910,삼프로TV_경제의신과함께,2021090800,[증시 염탐정] 약보합으로 출발한 시장.. 대형주의 방향은? f.이베스트투자증권 염...,자 그러면 오늘 또 증시 개장 상 1정리 해 주시아 염수 아니라 니모 지겠습니다 어...,100475,1813,2021-09-08,00,-1
1,LG화학,51910,삼프로TV_경제의신과함께,2021090800,"[뉴스3] ""美증시 비중축소""…잇단 비관론에 다우 270p↓",자 오늘 9월 8일 수요일 순서와먼저 뉴스 3부터 내 무게 좀 정리를해 드릴 텐데 ...,15483,1348,2021-09-08,00,-1
2,LG화학,51910,삼프로TV_경제의신과함께,2021090800,"[글로벌마켓 브리핑] 스가 총리 퇴진 발표 이후, 일본 증시 상승 랠리! f. 미래...",여러분 안녕하세요 광고 보고 왔습니다오늘 힘차게 한번 시작해 보도록하겠습니다 언제나...,7231,1711,2021-09-08,00,-1
3,LG화학,51910,삼프로TV_경제의신과함께,2021090800,[증시 셔터맨] 외인과 기관은 왜 삼성전자와 카카오뱅크를 매도할까? f. 한국투자증...,4 아 오늘한국투자증권 에서 오셨죠 우리박지영 차장을 모시겠습니다 어서오십시오 네 ...,21472,1511,2021-09-08,00,-1
4,LG화학,51910,삼프로TV_경제의신과함께,2021090800,[증시 셔터맨] 외인과 기관은 왜 삼성전자와 카카오뱅크를 매도할까? f. 한국투자증...,4 아 오늘한국투자증권 에서 오셨죠 우리박지영 차장을 모시겠습니다 어서오십시오 네 ...,21472,1511,2021-09-08,00,-1
5,LG화학,51910,삼프로TV_경제의신과함께,2021090800,"[글로벌마켓 브리핑] 스가 총리 퇴진 발표 이후, 일본 증시 상승 랠리! f. 미래...",여러분 안녕하세요 광고 보고 왔습니다오늘 힘차게 한번 시작해 보도록하겠습니다 언제나...,7231,1711,2021-09-08,00,-1
6,LG화학,51910,삼프로TV_경제의신과함께,2021090800,[증시 염탐정] 약보합으로 출발한 시장.. 대형주의 방향은? f.이베스트투자증권 염...,자 그러면 오늘 또 증시 개장 상 1정리 해 주시아 염수 아니라 니모 지겠습니다 어...,100475,1813,2021-09-08,00,-1
7,LG화학,51910,삼프로TV_경제의신과함께,2021090800,"[뉴스3] ""美증시 비중축소""…잇단 비관론에 다우 270p↓",자 오늘 9월 8일 수요일 순서와먼저 뉴스 3부터 내 무게 좀 정리를해 드릴 텐데 ...,15483,1348,2021-09-08,00,-1


In [141]:
## test

print(lgc_210908['Contents_input'][2][:20])

for i in range(len(lgc_210908['Contents_input'])):
    
    media = ['오늘', '년도', '이제', '요즘', '지금', '우리나라', '한번', '사실', '처럼', \
             '증시', '개장', '얘기', '이렇다', '참고', '없다', '때문', '아니다', \
             '안녕하십니까', '안녕하세요우', '안녕하다', '어서', '오십', '시오', '모습', '내용', '정도', \
             '그렇다', '어쨌든', '어떻다', '박지영', '미래에셋', '어머님', '음악']

    for word in lgc_210908['Contents_input'][i]:
#             if word in stopwords:
#                 lgc_210908['Contents_input'][i].remove(word)
        if word in media:
            lgc_210908['Contents_input'][i].remove(word)
                
print(lgc_210908['Contents_input'][2][:20])

['광고', '힘차다', '시작', '증권', '의장', '선임', '전이', '매년', '안녕하세요우', '난수', '간이', '럼또뭡', '사과', '감히', '방소', '점점', '덥다', '고합', '다정', '말로']
['광고', '힘차다', '시작', '증권', '의장', '선임', '전이', '매년', '난수', '간이', '럼또뭡', '사과', '감히', '방소', '점점', '덥다', '고합', '다정', '말로', '사과']


In [143]:
%%time

lgc_210908 = youtube_pre(lgc_210908_bf)

데이터 셋 구성 완료
okt 모듈 활성
태그 삭제 및 품사 태깅 완료
명사, 형용사 추출
불용어 사전 로딩..
불용어 처리 완료
단어 길이 1이하인 단어 제거
Wall time: 2min 31s


In [145]:
print(lgc_210908['Contents_input'][2][:20])

['광고', '힘차다', '시작', '미래에셋', '증권', '의장', '선임', '전이', '매년', '안녕하다', '안녕하세요우', '난수', '간이', '럼또뭡', '사과', '감히', '방소', '점점', '덥다', '고합']


In [132]:
lgc_210908.head()

,st_n,st_cd,ch_nm,date,title,text,views,length,date_d,date_h,ud_x,ud_y,Contents_input
0,LG화학,51910,삼프로TV_경제의신과함께,2021090800,[증시 염탐정] 약보합으로 출발한 시장.. 대형주의 방향은? f.이베스트투자증권 염...,자그러면오늘또증시개장상1정리해주시아염수아니라니모지겠습니다어서오십시오네마같습니까자우리...,100475,1813,2021-09-08,00,-1,-1,"[정리, 염수, 니모, 오십, 네마, 하든, 일로, 모습, 모집, 노래, 아이오와,..."
1,LG화학,51910,삼프로TV_경제의신과함께,2021090800,"[뉴스3] ""美증시 비중축소""…잇단 비관론에 다우 270p↓",자오늘9월8일수요일순서와먼저뉴스3부터내무게좀정리를해드릴텐데예아무래도미국소식도좀갤주너...,15483,1348,2021-09-08,00,-1,-1,"[무게, 정리, 미국, 소식, 주너, 미국, 시장, 음소, 간밤, 진행, 좋다, 아..."
2,LG화학,51910,삼프로TV_경제의신과함께,2021090800,"[글로벌마켓 브리핑] 스가 총리 퇴진 발표 이후, 일본 증시 상승 랠리! f. 미래...",여러분안녕하세요광고보고왔습니다오늘힘차게한번시작해보도록하겠습니다언제나처럼미래에셋증권의...,7231,1711,2021-09-08,00,-1,-1,"[안녕하다, 광고, 힘차다, 시작, 처럼, 증권, 의장, 선임, 전이, 매년, 안녕..."
3,LG화학,51910,삼프로TV_경제의신과함께,2021090800,[증시 셔터맨] 외인과 기관은 왜 삼성전자와 카카오뱅크를 매도할까? f. 한국투자증...,4아오늘한국투자증권에서오셨죠우리박지영차장을모시겠습니다어서오십시오네안녕하십니까4한국투...,21472,1511,2021-09-08,00,-1,-1,"[한국, 투자, 증권, 박지영, 차장, 시오, 안녕하십니까, 한국, 투자, 증권, ..."
4,LG화학,51910,삼프로TV_경제의신과함께,2021090800,[증시 셔터맨] 외인과 기관은 왜 삼성전자와 카카오뱅크를 매도할까? f. 한국투자증...,4아오늘한국투자증권에서오셨죠우리박지영차장을모시겠습니다어서오십시오네안녕하십니까4한국투...,21472,1511,2021-09-08,00,-1,-1,"[한국, 투자, 증권, 박지영, 차장, 시오, 안녕하십니까, 한국, 투자, 증권, ..."


In [126]:
lgc_210908.Contents_input

0    [정리, 염수, 니모, 오십, 네마, 하든, 일로, 모습, 모집, 노래, 아이오와,...
1    [무게, 정리, 미국, 소식, 주너, 미국, 시장, 음소, 간밤, 진행, 좋다, 아...
2    [안녕하다, 광고, 힘차다, 시작, 처럼, 증권, 의장, 선임, 전이, 매년, 안녕...
3    [한국, 투자, 증권, 박지영, 차장, 시오, 안녕하십니까, 한국, 투자, 증권, ...
4    [한국, 투자, 증권, 박지영, 차장, 시오, 안녕하십니까, 한국, 투자, 증권, ...
5    [안녕하다, 광고, 힘차다, 시작, 처럼, 증권, 의장, 선임, 전이, 매년, 안녕...
6    [정리, 염수, 니모, 오십, 네마, 하든, 일로, 모습, 모집, 노래, 아이오와,...
7    [무게, 정리, 미국, 소식, 주너, 미국, 시장, 음소, 간밤, 진행, 좋다, 아...
Name: Contents_input, dtype: object

In [127]:
lgc_210908.Contents_input[0]

['정리',
 '염수',
 '니모',
 '오십',
 '네마',
 '하든',
 '일로',
 '모습',
 '모집',
 '노래',
 '아이오와',
 '따갑다',
 '달라',
 '위험부담',
 '예맥',
 '강하다',
 '요청',
 '김택진',
 '진영',
 '머신',
 '매트',
 '개장',
 '상황',
 '은행',
 '관심',
 '좋다',
 '금융',
 '재정',
 '옥애널',
 '리스트',
 '은행',
 '업종',
 '간격',
 '보고',
 '간단',
 '월동',
 '외국인',
 '매도',
 '계속',
 '코스피',
 '은행',
 '수급',
 '무료',
 '좋다',
 '목요일',
 '그다음',
 '긴축',
 '속도',
 '경기',
 '확장',
 '북면',
 '연장',
 '가능성',
 '다시',
 '경기',
 '금리',
 '베팅',
 '실적',
 '상황',
 '서주',
 '당대',
 '금도',
 '가능성',
 '투자',
 '수요',
 '증가',
 '가능성',
 '고합',
 '다그',
 '래서',
 '변경',
 '은행',
 '전체',
 '순위',
 '좋다',
 '작년',
 '대비',
 '가내',
 '순위',
 '증가',
 '예상',
 '그다음',
 '올해',
 '예상',
 '배당',
 '수익률',
 '투자',
 '행주',
 '연말',
 '은행',
 '대해',
 '관심',
 '좀뭐',
 '리스크',
 '만화',
 '접근',
 '요좀',
 '해주시',
 '좋다',
 '그다음',
 '증권',
 '재운',
 '이분',
 '굉장하다',
 '피트',
 '코인',
 '보고서',
 '비트코인',
 '가기',
 '급락',
 '최근',
 '설번',
 '거기',
 '법정',
 '놈',
 '개널',
 '비트코인',
 '법정',
 '통화',
 '채택',
 '예고',
 '살바도르',
 '달러',
 '버팔로',
 '서치',
 '인정',
 '시작',
 '제품',
 '구매',
 '세금',
 '납부',
 '비트코인',
 '자국',
 '국민',
 '혼란',
 '위해',
 '비트코인',
 '달러',


In [128]:
lgc_210908_list = []

for i in lgc_210908.Contents_input:
    for j in i:
        lgc_210908_list.append(j)
    
lgc_210908_list

['정리',
 '염수',
 '니모',
 '오십',
 '네마',
 '하든',
 '일로',
 '모습',
 '모집',
 '노래',
 '아이오와',
 '따갑다',
 '달라',
 '위험부담',
 '예맥',
 '강하다',
 '요청',
 '김택진',
 '진영',
 '머신',
 '매트',
 '개장',
 '상황',
 '은행',
 '관심',
 '좋다',
 '금융',
 '재정',
 '옥애널',
 '리스트',
 '은행',
 '업종',
 '간격',
 '보고',
 '간단',
 '월동',
 '외국인',
 '매도',
 '계속',
 '코스피',
 '은행',
 '수급',
 '무료',
 '좋다',
 '목요일',
 '그다음',
 '긴축',
 '속도',
 '경기',
 '확장',
 '북면',
 '연장',
 '가능성',
 '다시',
 '경기',
 '금리',
 '베팅',
 '실적',
 '상황',
 '서주',
 '당대',
 '금도',
 '가능성',
 '투자',
 '수요',
 '증가',
 '가능성',
 '고합',
 '다그',
 '래서',
 '변경',
 '은행',
 '전체',
 '순위',
 '좋다',
 '작년',
 '대비',
 '가내',
 '순위',
 '증가',
 '예상',
 '그다음',
 '올해',
 '예상',
 '배당',
 '수익률',
 '투자',
 '행주',
 '연말',
 '은행',
 '대해',
 '관심',
 '좀뭐',
 '리스크',
 '만화',
 '접근',
 '요좀',
 '해주시',
 '좋다',
 '그다음',
 '증권',
 '재운',
 '이분',
 '굉장하다',
 '피트',
 '코인',
 '보고서',
 '비트코인',
 '가기',
 '급락',
 '최근',
 '설번',
 '거기',
 '법정',
 '놈',
 '개널',
 '비트코인',
 '법정',
 '통화',
 '채택',
 '예고',
 '살바도르',
 '달러',
 '버팔로',
 '서치',
 '인정',
 '시작',
 '제품',
 '구매',
 '세금',
 '납부',
 '비트코인',
 '자국',
 '국민',
 '혼란',
 '위해',
 '비트코인',
 '달러',


In [129]:
len(lgc_210908_list)

9458

In [130]:
count = Counter(lgc_210908_list)
words = dict(count.most_common())

words_df = pd.DataFrame.from_dict([words]).T.reset_index()
words_df.columns = ['words', 'freq']

In [131]:
words_df

,words,freq
0,좋다,124
1,중국,102
2,이제,78
3,기업,56
4,굉장하다,54
...,...,...
2246,요,2
2247,키스,2
2248,깔끔하다,2
2249,반값,2


In [123]:
words_df.to_csv('lgc_210908_wc.csv', header=True, index=False, encoding = 'utf-8-sig')